In [1]:
from cafecosmos import World, Player

/Users/nico/.virtualenvs/woww/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
abi_dir = "./cafecosmos-contracts/abi"
rpc = "https://rpc.garnetchain.com"
world_address = "0x0209FE2b6C1a2B8cE12D8d9D6cf2D78a3Ff324FE"
indexer_url = "https://indexer.mud.garnetchain.com/q"
mud_config_path = "./cafecosmos-contracts/mud.config.ts"
block_explorer_url = "https://explorer.garnetchain.com"


cafecosmos = World(rpc, 
              world_address, 
              abi_dir, 
              indexer_url=indexer_url, 
              mud_config_path=mud_config_path,
              block_explorer_url=block_explorer_url)

player = Player(env_key_name="PLAYER1", world=cafecosmos, land_id=5)

In [3]:
player.craft_item("Furnace")

Adding batch call to craft item ID 987


AttributeError: 'str' object has no attribute 'hex'

In [6]:
player.cafecosmos.errors

{'7149a3c1': ('LandScenarioUserTestingSystem', 'EncodedLengths_InvalidLength'),
 '23230fa3': ('LandScenarioUserTestingSystem', 'Slice_OutOfBounds'),
 '7e8578d3': ('LandScenarioUserTestingSystem', 'Store_IndexOutOfBounds'),
 '31b46683': ('LandScenarioUserTestingSystem', 'Store_InvalidResourceType'),
 'a65010b4': ('LandScenarioUserTestingSystem', 'Store_InvalidSplice'),
 'a019e4da': ('RegistrationSystem', 'FieldLayout_Empty'),
 'eba964de': ('RegistrationSystem', 'FieldLayout_InvalidStaticDataLength'),
 '7237f237': ('RegistrationSystem',
  'FieldLayout_StaticLengthDoesNotFitInAWord'),
 '1d8d44db': ('RegistrationSystem', 'FieldLayout_StaticLengthIsNotZero'),
 '53b2dd05': ('RegistrationSystem', 'FieldLayout_StaticLengthIsZero'),
 '6793df62': ('RegistrationSystem', 'FieldLayout_TooManyDynamicFields'),
 'ecd893b9': ('RegistrationSystem', 'FieldLayout_TooManyFields'),
 '14012068': ('IWorld', 'Module_AlreadyInstalled'),
 '5f8b4887': ('IWorld', 'Module_MissingDependency'),
 'ea8b184c': ('IWorld'

In [3]:
player.display_land()

x,0,1,2,3,4,5,6,7,8,9
y,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,NaN,NaN,NaN,,NaN,NaN,NaN,,,
5,NaN,NaN,NaN,,NaN,,NaN,,,
6,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [4]:
player.display_inventory()

Icon,Item Name,Quantity
,Avocado,8
,Bananas,99
,Coffee,3
,Egg,26
,Raspberry,39
,Sugar,18
,Olives,14
,Bismuth,32
,Crystal,144
,Clay,32


In [5]:
# player.place_item(3, 5, "Purple Floor")

In [6]:
player.display_land()

x,0,1,2,3,4,5,6,7,8,9
y,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,NaN,NaN,NaN,,NaN,NaN,NaN,,,
5,NaN,NaN,NaN,,NaN,,NaN,,,
6,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [7]:
player.get_craftable()

,Item ID,Item Name,Max Craftable,Requirements
0,5,Coffee,45,1 x Coffee Bean
1,27,Glass,144,1 x Crystal
2,29,Blade,8,"4 x Bismuth, 1 x Crystal"
3,82,Cutting Board,44,"1 x Sticks, 1 x Crystal, 3 x Pink Wood"
4,90,Pink Chair,33,4 x Pink Wood
5,92,Green Table,7,"3 x Pink Wood, 2 x Purple Wood"
6,93,Purple Table,3,5 x Purple Wood
7,94,Pink Table,26,5 x Pink Wood
8,96,Fence,31,"5 x Sticks, 2 x Pink Wood"
9,97,Pink Floor,44,3 x Pink Wood


In [8]:
player.id_to_name(19)

'Bismuth'

In [9]:
player.id_to_name(25)

'Pink Wood'

In [11]:
player.id_to_name(20)

'Crystal'

In [ ]:
player.display_inventory()

Icon,Item Name,Quantity
,Avocado,8
,Bananas,99
,Coffee,3
,Egg,26
,Raspberry,39
,Sugar,18
,Olives,14
,Bismuth,32
,Crystal,144
,Clay,32
